# ST2195 Programming for data science
## 200615086, InJun Yeo

In [ ]:
import numpy as np
import pandas as pd
import random
from datetime import datetime, timedelta
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

In [ ]:
airports = pd.read_csv("./dataverse_files/airports.csv")
carriers = pd.read_csv("./dataverse_files/carriers.csv")
planes = pd.read_csv("./dataverse_files/plane-data.csv")

In [ ]:
flight_2005 = pd.read_csv("./dataverse_files/flights/2005.csv.bz2")
flight_2006 = pd.read_csv("./dataverse_files/flights/2006.csv.bz2")
flight_2007 = pd.read_csv("./dataverse_files/flights/2007.csv.bz2")

In [ ]:
flight = pd.concat([flight_2005, flight_2006, flight_2007])
flight.shape

In [ ]:
flight = flight[flight.Cancelled != 1]
flight.shape

In [ ]:
flight.corr()["ArrDelay"]["DepDelay"]
#0.9202856798860516

In [ ]:
x = flight["ArrDelay"][(flight["ArrDelay"] > 0) & (flight["DepDelay"] > 0)]
y = flight["DepDelay"][(flight["ArrDelay"] > 0) & (flight["DepDelay"] > 0)]

plt.figure(figsize=(12, 9))
plt.scatter(x, y, c = 'blue', alpha = 0.2)
plt.title("Scatter Plot (DepDelay & ArrDelay)", fontsize = 15)
plt.xlabel("Arrival Delay", fontsize = 12)
plt.ylabel("Departure Delay", fontsize = 12)
plt.show()

In [ ]:
flight = flight.dropna(subset = ["ArrDelay"])
flight = flight[flight.ArrDelay > 0]
flight.shape
#9714043

In [ ]:
flight['Delayed'] = flight['DepDelay'] > 15

In [ ]:
flight = flight.astype({'CRSDepTime':'int', 'CRSArrTime':'int'})

flight['DepHour'] = flight['CRSDepTime'] // 100
flight['DepMin'] = flight['CRSDepTime'] % 100
flight['ArrHour'] = flight['CRSArrTime'] // 100
flight['ArrMin'] = flight['CRSArrTime'] % 100

flight['DepDate'] = flight[['Year','Month','DayofMonth',
                            'DepHour', 'DepMin']].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
flight['ArrDate'] = flight[['Year','Month','DayofMonth',
                            'ArrHour', 'ArrMin']].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)

flight['DepDate'] = pd.to_datetime(flight['DepDate'], format="%Y %m %d %H %M")
flight['ArrDate'] = pd.to_datetime(flight['ArrDate'], format="%Y %m %d %H %M")

### 1. When is the best time of day, day of the week, and time of year to fly to minimise delays?

In [ ]:
group_by_hour = pd.DataFrame()

temp_grpbyhr = flight.groupby(['DepHour','Delayed'])
temp_grpbyhr = temp_grpbyhr.size().reset_index(name='Counts')

delayed = temp_grpbyhr[temp_grpbyhr.Delayed != False]['Counts']
delayed.index = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23]
group_by_hour['Delayed counts'] = delayed

total = temp_grpbyhr.groupby(temp_grpbyhr['DepHour'])['Counts'].sum()
group_by_hour['Total'] = total

group_by_hour['Proportion of Delayed'] = group_by_hour['Delayed counts'] / group_by_hour['Total'] * 100
group_by_hour['Mean DepDelay'] = flight.groupby('DepHour')['DepDelay'].mean()
group_by_hour.reset_index(drop = False, inplace = True)
group_by_hour.rename(columns={'index':'DepHour'},inplace=True)

In [ ]:
group_by_wday = pd.DataFrame()

temp_grpbywd = flight.groupby(['DayOfWeek','Delayed'])
temp_grpbywd = temp_grpbywd.size().reset_index(name='Counts')

delayed = temp_grpbywd[temp_grpbywd.Delayed != False]['Counts']
delayed.index = [1,2,3,4,5,6,7]
group_by_wday['Delayed counts'] = delayed

total = temp_grpbywd.groupby(temp_grpbywd['DayOfWeek'])['Counts'].sum()
group_by_wday['Total'] = total

group_by_wday['Proportion of Delayed'] = group_by_wday['Delayed counts'] / group_by_wday['Total'] * 100
group_by_wday['Mean DepDelay'] = flight.groupby('DayOfWeek')['DepDelay'].mean()
group_by_wday.reset_index(drop = False, inplace = True)
group_by_wday.rename(columns={'index':'DayOfWeek'},inplace=True)
group_by_wday['DayOfWeek'] = ["Mon","Tue","Wed","Thu","Fri","Sat","Sun"]
group_by_wday = group_by_wday.reindex(index=[6,0,1,2,3,4,5])

In [ ]:
group_by_month = pd.DataFrame()

temp_grpbymth = flight.groupby(['Month','Delayed'])
temp_grpbymth = temp_grpbymth.size().reset_index(name='Counts')

delayed = temp_grpbymth[temp_grpbymth.Delayed != False]['Counts']
delayed.index = [1,2,3,4,5,6,7,8,9,10,11,12]
group_by_month['Delayed counts'] = delayed

total = temp_grpbymth.groupby(temp_grpbymth['Month'])['Counts'].sum()
group_by_month['Total'] = total

group_by_month['Proportion of Delayed'] = group_by_month['Delayed counts'] / group_by_month['Total'] * 100
group_by_month['Mean DepDelay'] = flight.groupby('Month')['DepDelay'].mean()
group_by_month.reset_index(drop = False, inplace = True)
group_by_month.rename(columns={'index':'Month'},inplace=True)
group_by_month['Month'] = ["Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"]

In [ ]:
x = range(len(group_by_hour['DepHour']))
y1 = group_by_hour['Proportion of Delayed']
y2 = group_by_hour['Mean DepDelay']

fig, ax1 = plt.subplots(figsize=(12, 9))

ax1.bar(x, y1, color='deeppink', label="Delayed rate", alpha=0.5, width=0.7)
ax1.set_title("Group by Hour", fontsize = 15)
ax1.set_xlabel("Hour")
ax1.set_ylabel("Delayed rate (%)")
plt.xticks(x, group_by_hour['DepHour'])

ax2 = ax1.twinx()

ax2.plot(x, y2, '-o', color='red', markersize=3, linewidth=2, alpha=0.7, label="DepDelay")
ax2.set_ylabel("DepDelay (min)")

ax1.legend(loc='upper right')
ax2.legend(loc='upper right', bbox_to_anchor=(1, 0.95))

plt.show()

In [ ]:
x = range(len(group_by_wday['DayOfWeek']))
y1 = group_by_wday['Proportion of Delayed']
y2 = group_by_wday['Mean DepDelay']

fig, ax1 = plt.subplots(figsize=(12, 9))

ax1.bar(x, y1, color='deeppink', label="Delayed rate", alpha=0.5, width=0.7)
ax1.set_title("Group by Day of Week", fontsize = 15)
ax1.set_xlabel("Day of Week")
ax1.set_ylabel("Delayed rate (%)")
plt.xticks(x, group_by_wday['DayOfWeek'])

ax2 = ax1.twinx()

ax2.plot(x, y2, '-o', color='red', markersize=3, linewidth=2, alpha=0.7, label="DepDelay")
ax2.set_ylabel("DepDelay (min)")

ax1.legend(loc='upper right')
ax2.legend(loc='upper right', bbox_to_anchor=(1, 0.95))

plt.show()

In [ ]:
x = range(len(group_by_month['Month']))
y1 = group_by_month['Proportion of Delayed']
y2 = group_by_month['Mean DepDelay']

fig, ax1 = plt.subplots(figsize=(12, 9))

ax1.bar(x, y1, color='deeppink', label="Delayed rate", alpha=0.5, width=0.7)
ax1.set_title("Group by Month", fontsize = 15)
ax1.set_xlabel("Month")
ax1.set_ylabel("Delayed rate (%)")
plt.xticks(x, group_by_month['Month'])

ax2 = ax1.twinx()

ax2.plot(x, y2, '-o', color='red', markersize=3, linewidth=2, alpha=0.7, label="DepDelay")
ax2.set_ylabel("DepDelay (min)")

ax1.legend(loc='upper right')
ax2.legend(loc='upper right', bbox_to_anchor=(1, 0.95))

plt.show()

## 2. Do older planes suffer more delays?

In [ ]:
flight_q2 = pd.merge(flight,planes, how='left', left_on="TailNum", right_on="tailnum")
flight_q2 = flight_q2[["year","CarrierDelay"]]
flight_q2.rename(columns={'year':'Manufacture Year'},inplace=True)
flight_q2 = flight_q2.dropna()

In [ ]:
flight_q2['Manufacture Year'].unique()

In [ ]:
non_year = flight_q2[(flight_q2['Manufacture Year'] == 'None')|(flight_q2['Manufacture Year'] == '0000')].index
flight_q2.drop(non_year, inplace=True)
flight_q2['Manufacture Year'].unique()

In [ ]:
flight_q2 = flight_q2.astype({'Manufacture Year':'int'})

In [ ]:
x = flight_q2['Manufacture Year']
y = flight_q2['CarrierDelay']
year_order = list(range(flight_q2['Manufacture Year'].min(), flight_q2['Manufacture Year'].max()))

plt.figure(figsize=(12, 9))
plt.scatter(x, y, c = 'green', alpha = 0.5)
plt.xticks(year_order, rotation=90)
plt.title("Correlation between Manufacture Year and Carrier Delay", fontsize = 15)
plt.xlabel("Manufacture Year", fontsize = 12)
plt.ylabel("Carrier Delay (min)", fontsize = 12)
plt.show()

In [ ]:
flight_q2.groupby(['Manufacture Year']).count()

In [ ]:
flight_q2.corr()["CarrierDelay"]["Manufacture Year"]
#0.0016321609213292645

## 3. How does the number of people flying between different locations change over time?

In [ ]:
migration_2005 = airports[["iata","state"]]
migration_2006 = airports[["iata","state"]]
migration_2007 = airports[["iata","state"]]

In [ ]:
route_2005 = flight[flight['Year'] == 2005]
route_2005 = route_2005.loc[:,['Origin','Dest']]
route_2005 = route_2005.value_counts().reset_index(name='counts')

route_2006 = flight[flight['Year'] == 2006]
route_2006 = route_2006.loc[:,['Origin','Dest']]
route_2006 = route_2006.value_counts().reset_index(name='counts')

route_2007 = flight[flight['Year'] == 2007]
route_2007 = route_2007.loc[:,['Origin','Dest']]
route_2007 = route_2007.value_counts().reset_index(name='counts')

In [ ]:
migration_2005 = pd.merge(migration_2005, route_2005.groupby(['Dest']).sum(), how='left', left_on="iata", right_on="Dest")
migration_2005 = migration_2005.rename(columns={'counts':'outflow'})
migration_2005 = pd.merge(migration_2005, route_2005.groupby(['Origin']).sum(), how='left', left_on="iata", right_on="Origin")
migration_2005 = migration_2005.rename(columns={'counts':'inflow'})
migration_2005 = migration_2005.loc[migration_2005[["outflow","inflow"]].dropna(how='all').index]
migration_2005['Total flights'] = migration_2005['outflow'] + migration_2005['inflow']
migration_2005 = migration_2005.groupby('state').sum()
migration_2005 = migration_2005.sort_values('Total flights', ascending=False)

migration_2006 = pd.merge(migration_2006, route_2006.groupby(['Dest']).sum(), how='left', left_on="iata", right_on="Dest")
migration_2006 = migration_2006.rename(columns={'counts':'outflow'})
migration_2006 = pd.merge(migration_2006, route_2006.groupby(['Origin']).sum(), how='left', left_on="iata", right_on="Origin")
migration_2006 = migration_2006.rename(columns={'counts':'inflow'})
migration_2006 = migration_2006.loc[migration_2006[["outflow","inflow"]].dropna(how='all').index]
migration_2006['Total flights'] = migration_2006['outflow'] + migration_2006['inflow']
migration_2006 = migration_2006.groupby('state').sum()
migration_2006 = migration_2006.sort_values('Total flights', ascending=False)

migration_2007 = pd.merge(migration_2007, route_2007.groupby(['Dest']).sum(), how='left', left_on="iata", right_on="Dest")
migration_2007 = migration_2007.rename(columns={'counts':'outflow'})
migration_2007 = pd.merge(migration_2007, route_2007.groupby(['Origin']).sum(), how='left', left_on="iata", right_on="Origin")
migration_2007 = migration_2007.rename(columns={'counts':'inflow'})
migration_2007 = migration_2007.loc[migration_2007[["outflow","inflow"]].dropna(how='all').index]
migration_2007['Total flights'] = migration_2007['outflow'] + migration_2007['inflow']
migration_2007 = migration_2007.groupby('state').sum()
migration_2007 = migration_2007.sort_values('Total flights', ascending=False)

In [ ]:
print('Total flights in 2005 is', migration_2005['Total flights'].sum())
print('Total flights in 2006 is', migration_2006['Total flights'].sum())
print('Total flights in 2007 is', migration_2007['Total flights'].sum())

In [ ]:
ratio05 = migration_2005['Total flights']
labels05 = migration_2005.index
ratio06 = migration_2006['Total flights']
labels06 = migration_2006.index
ratio07 = migration_2007['Total flights']
labels07 = migration_2007.index
wedgeprops={'width': 0.7, 'edgecolor': 'w', 'linewidth': 5}


fig, ax = plt.subplots(1, 3, figsize=(18, 6))

ax[0].pie(ratio05, labels=labels05, startangle=90, counterclock=False, 
          textprops={'fontsize': 12}, 
          autopct=lambda p: format(p, '.2f') if p > 5 else None, wedgeprops=wedgeprops)
ax[0].set_title("2005", fontsize=15, weight='bold')

ax[1].pie(ratio06, labels=labels06, startangle=90, counterclock=False, 
          textprops={'fontsize': 12}, 
          autopct=lambda p: format(p, '.2f') if p > 5 else None, wedgeprops=wedgeprops)
ax[1].set_title("2006", fontsize=15, weight='bold')

ax[2].pie(ratio07, labels=labels07, startangle=90, counterclock=False, 
          textprops={'fontsize': 12}, 
          autopct=lambda p: format(p, '.2f') if p > 5 else None, wedgeprops=wedgeprops)
ax[2].set_title("2007", fontsize=15, weight='bold')

plt.show()

## 4. Can you detect cascading failures as delays in one airport create delays in others?

In [ ]:
flight_q4 = flight[['DepDate', 'ArrDate',
                    'ArrDelay', 'LateAircraftDelay',
                    'Origin', 'Dest']]
flight_q4

In [ ]:
flight_q4 = flight_q4.drop(flight_q4[flight_q4['DepDate'] > flight_q4['ArrDate']].index)
flight_q4 = flight_q4.sort_values('DepDate')
flight_q4

In [ ]:
LAD = flight_q4
LAD['LateAircraftDelay'] = LAD['LateAircraftDelay'].replace(0, np.NaN)
LAD = LAD.dropna()
LAD = LAD.sort_values('ArrDate')
LAD

In [ ]:
cf_count = 0
for k in range(0,LAD.shape[0]):
    flight_q4.next_flight = (LAD.iloc[k,1] < flight_q4.DepDate) & (LAD.iloc[k,1] + timedelta(minutes=LAD.iloc[k,2]+10) > flight_q4.DepDate)
    flight_q4.same_airport = LAD.iloc[k,5] == flight_q4.Origin
    if (flight_q4.next_flight & flight_q4.same_airport).sum() > 0:
        if flight_q4[flight_q4.next_flight & flight_q4.same_airport].iloc[0,2] > 0:
            cf_count = cf_count + 1
cf_count

In [ ]:
print("The probability of the cascading failures is", round(cf_count / LAD.shape[0] * 100,2),"%")

#87075/289119*100 = 30.12%
#93860/307255*100 = 30.55%
#104899/337415*100 = 31.09%
#125197/400324*100 = 31.27%

## 5. Use the available variables to construct a model that predicts delays.

In [ ]:
random_index = random.sample(range(flight.shape[0]), 5000)
flight_q5 = flight.reset_index()
flight_q5 = flight_q5.loc[random_index]
flight_q5

In [ ]:
ArrDelay = flight_q5['ArrDelay']

In [ ]:
flight_q5 = pd.merge(flight_q5, planes, left_on = 'TailNum', right_on = 'tailnum')

In [ ]:
flight_q5 = flight_q5[['Year','Month','DayofMonth','DayOfWeek','TailNum','year']]

In [ ]:
flight_q5 = flight_q5[flight_q5.year != 'None'].dropna()

In [ ]:
LRegression = LinearRegression()

In [ ]:
enc = OneHotEncoder()
enc_flight_q5 = pd.DataFrame(enc.fit_transform(flight_q5[['TailNum']]).toarray())
flight_q5 = flight_q5.join(enc_flight_q5)

In [ ]:
flight_q5 = flight_q5.drop('TailNum', 1)

In [ ]:
sample = flight_q5.dropna().head(2000)
predict_y = ArrDelay.dropna().head(2000)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(sample, predict_y, test_size=0.3, shuffle=True, random_state=42)

In [ ]:
LRegression.fit(x_train, y_train)

In [ ]:
y_hat = LRegression.predict(x_test)
y_hat

In [ ]:
y_test = y_test.reset_index()
y_test["y_hat"] = pd.DataFrame({"y_hat":y_hat})
predict_evaluation = y_test[['ArrDelay','y_hat']]
predict_evaluation

In [ ]:
predict_evaluation.corr()["ArrDelay"]["y_hat"]